In [ ]:
# load env variables from .env file
from dotenv import load_dotenv
import os
load_dotenv()


In [ ]:

chatgpt_wed_api_key = os.environ.get("chatgpt_wed_api_key")
chatgpt_wed_region = os.environ.get("chatgpt_wed_region")
chatgpt_wed_endpoint = os.environ.get("chatgpt_wed_endpoint")
chatgpt_wed_api_version = os.environ.get("chatgpt_wed_api_version")
chatgpt_wed_deployment = os.environ.get("chatgpt_wed_deployment")
search_wed_endpoint = os.environ.get("search_wed_endpoint")
search_wed_key = os.environ.get("search_wed_key")
search_wed_index = os.environ.get("search_wed_index")

In [ ]:
from urllib.parse import urljoin, urlparse 
aoai_extension_endpoint = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/extensions/chat/completions?api-version={chatgpt_wed_api_version}")
# chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgot_wed_deployment}/completions?api-version={chatgpt_wed_api_version}")
chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/chat/completions?api-version=2023-03-15-preview")


In [ ]:
print(f"AOAI Endpoint: {aoai_extension_endpoint}")
print(f"ChatGPT Endpoint: {chatgpt_api_url}")

In [ ]:
import json 

def fprint(obj):
    # cast obj to dict before printing



    print(json.dumps(dict(obj), indent=4))


messages = { 
    "messages": [
        {
            "role": "system",
		    "content": """Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. 
Be brief in your answers.
Say goodbye at the end of every response. 
            """
        }, 
        {
            "role": "user", 
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        }

    ]
}


##TODO:  Include feilds mapping here
datasources = {
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_wed_endpoint,
                "key": search_wed_key,
                "indexName": search_wed_index,
                "fieldsMapping": {
                    "titleField": "sourcefile",
                    "urlField": "sourcepage",
                    "filepathField": "sourcepage",
                    "contentFields": [
                        "content"
                    ],
                    "contentFieldsSeparator": "\n"
                },
                "inScope": "true",
                "topNDocuments": 3,
                "queryType": "semantic",
                "semanticConfiguration": "default",
                "roleInformation": ""
            }
        }
    ]
}


headers = {
    "Content-Type": "application/json",
    "api-key": f"{chatgpt_wed_api_key}",
    "chatgpt_url": chatgpt_api_url, 
    "chatgpt_key": chatgpt_wed_api_key,
}



In [ ]:
print(headers.keys())

In [ ]:

# combine datasources and messages into a payload
payload = {**datasources, **messages}
#fprint(payload)


In [ ]:
import requests
response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))
fprint(response.json())


In [ ]:
# read response as dict 
response_dict = response.json()

response_dict["choices"][0]['messages']

for m in response_dict["choices"][0]['messages']: 
    if m["role"] == "assistant":
        messages["messages"].append(m)
    

In [ ]:
fprint(messages)


In [ ]:
messages["messages"].append({
    "role": "user",
    "content": "tell me about northwind health plan"
})

fprint(messages)

response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))


In [ ]:
len(response.json()['choices'])

In [ ]:
fprint(response.json()['choices'][0])

In [ ]:
# read citations from response 
citations = []
intent = []
for m in response.json()['choices'][0]['messages']: 
    if m["role"] == "tool":
        citations = json.loads(m["content"])['citations']
        intent = json.loads(m["content"])['intent']

    if m["role"] == "assistant":
        messages["messages"].append(m)

print(len(citations))

In [ ]:
for c in citations:
    fprint(c)

In [ ]:
print(intent)